In [1]:
from graphblas import monoid
from graphblas.semiring import any_pair, plus_first

from .boundary import edge_boundary, node_boundary


ModuleNotFoundError: No module named 'graphblas'

kron_khatri_rao.ipynb

https://github.com/python-graphblas/graphblas-algorithms/blob/main/graphblas_algorithms/algorithms/cuts.py

Use these algorithms to find cut-related quantities:

In [ ]:

def cut_size(G, S, T=None, *, is_weighted=False):
    edges = edge_boundary(G, S, T, is_weighted=is_weighted)
    if is_weighted:
        rv = edges.reduce_scalar(monoid.plus).get(0)
    else:
        rv = edges.nvals
    if G.is_directed():
        edges = edge_boundary(G, T, S, is_weighted=is_weighted)
        if is_weighted:
            rv += edges.reduce_scalar(monoid.plus).get(0)
        else:
            rv += edges.nvals
    return rv


def volume(G, S, *, weighted=False):
    if weighted:
        degrees = plus_first(G._A @ S)
    else:
        degrees = G.get_property("row_degrees+", mask=S.S)
    return degrees.reduce(monoid.plus).get(0)


def normalized_cut_size(G, S, T=None):
    num_cut_edges = cut_size(G, S, T)
    volume_S = volume(G, S)
    volume_T = volume(G, T)
    return num_cut_edges * ((1 / volume_S) + (1 / volume_T))


def conductance(G, S, T=None):
    num_cut_edges = cut_size(G, S, T)
    volume_S = volume(G, S)
    volume_T = volume(G, T)
    return num_cut_edges / min(volume_S, volume_T)


def edge_expansion(G, S, T=None):
    num_cut_edges = cut_size(G, S, T)
    if T is None:
        Tnvals = S.size - S.nvals
    else:
        Tnvals = T.nvals
    return num_cut_edges / min(S.nvals, Tnvals)


def mixing_expansion(G, S, T=None):
    num_cut_edges = cut_size(G, S, T)
    return num_cut_edges / G._A.nvals  # Why no factor of 2 in denominator?


def node_expansion(G, S):
    neighborhood = any_pair(S @ G._A)
    return neighborhood.nvals / S.nvals


def boundary_expansion(G, S):
    result = node_boundary(G, S)
    return result.nvals / S.nvals

In [3]:
# https://github.com/python-graphblas/graphblas-algorithms/blob/main/graphblas_algorithms/algorithms/centrality/eigenvector.py

from graphblas import Vector

from graphblas_algorithms.algorithms._helpers import is_converged, normalize
from graphblas_algorithms.algorithms.exceptions import (
    ConvergenceFailure,
    GraphBlasAlgorithmException,
    PointlessConcept,
)

__all__ = ["eigenvector_centrality"]


def eigenvector_centrality(G, max_iter=100, tol=1.0e-6, nstart=None, name="eigenvector_centrality"):
    N = len(G)
    if N == 0:
        raise PointlessConcept("cannot compute centrality for the null graph")
    x = Vector(float, N, name="x")
    if nstart is None:
        x << 1.0 / N
    else:
        x << nstart
        denom = x.reduce().get(0)  # why not use L2 norm?
        if denom == 0:
            raise GraphBlasAlgorithmException("initial vector cannot have all zero values")
        x *= 1.0 / denom

    # Power iteration: make up to max_iter iterations
    A = G._A
    xprev = Vector(float, N, name="x_prev")
    for _ in range(max_iter):
        xprev << x
        x += x @ A
        normalize(x, "L2")
        if is_converged(xprev, x, tol):  # sum(abs(xprev - x)) < N * tol
            x.name = name
            return x
    raise ConvergenceFailure(max_iter)
    

ModuleNotFoundError: No module named 'graphblas'

Code the graph operators (line graph, clique graph) in graphblas 

In [2]:
#https://github.com/python-graphblas/graphblas-algorithms/blob/main/graphblas_algorithms/algorithms/simple_paths.py
import graphblas
from graphblas import Matrix, binary

__all__ = ["is_simple_path"]


def is_simple_path(G, nodes):
    if len(nodes) == 0:
        return False
    if len(nodes) == 1:
        return nodes[0] in G
    A = G._A
    if A.nvals < len(nodes) - 1:
        return False
    key_to_id = G._key_to_id
    indices = [key_to_id[key] for key in nodes if key in key_to_id]
    if len(indices) != len(nodes) or len(indices) > len(set(indices)):
        return False
    # Check all steps in path at once
    P = Matrix.from_coo(indices[:-1], indices[1:], True, nrows=A.nrows, ncols=A.ncols)
    P << binary.second(A & P)
    return P.nvals == len(indices) - 1

ModuleNotFoundError: No module named 'graphblas'

graphblas notes:

first: add, second:multiply

AX = b over tropical semiring solution code

The positive determinant of matrix A is $|A|^+$ and is defined by $|A|^+ = \sum_{ \sigma \in \mathcal{A}_n} \Pi_{i=1}^n a_{i \sigma(i)}$ and $|A|^- = \sum_{\sigma \mathcal{S}_n \setminus \mathcal{A}_n} \Pi_{i=1}^n a_{i \sigma(i)}$. 

The $\epsilon$-determinant of $A$ can be rewritten as $det_{\epsilon}(A) = |A|^+ + \epsilon( |A|^-)$. 

Def. The $\epsilon$-determinant of $A$ is $det_{\epsilon}(A) = \sum_{\sigma \in S_n} \epsilon^{\tau(\sigma)}( a_{1 \sigma(1)} \cdots a_{ n \sigma(n)})$ which is also equal to $|A|^+ + \epsilon(|A|^-)$ 

Thm 4:solution $X* = (A^- b) = (d^{-1} d_1, d^{-1} d_2, \cdots, d^{-1} d_n)^T$ 
where $d = det_{eps}(A)$ and $d_j = det_{eps}(A_{[j]}), j \in [n]$. 

In [ ]:
# pseudo inverse of system matrix (sec 4.1)


def pos_det(A):
    
def neg_det(A):
    
def det_eps(A, eps):
    return pos_det(A) + eps*neg_det(A)

def submat(A, i, j):
    Ai = np.delete(A, (i), axis=0)
    Aij = np.delete(Ai, (j), axis=1)
    return Aij

def adj_eps(A, eps): # eps-adjoint matrix
    # (n-1)(n-1) submatrix of A remove i-th row and j-th column 
    A_n1 = submat(A, i, j)
    eps_ij = np.pow(eps, i+j)
    prod= eps_ij*A_n1
    return np.transpose(prod)

def pseudo_inv(A, eps):
    A_deteps = det_eps(A, eps)
    A_deteps_inv = np.inv(A_deteps)
    return np.matmul(A_deteps_inv, adj_eps(A, eps))
    
# solution is A^- * b where A^- is pseudoinverse

In [ ]:
# from thm 4


code graph computation properties (based on shortest distance -> reciprocal deg distance) based on graphblas (in python?)

Then plugin the properties from graphblas into grakel ? 

min cycle basis over tropical semiring -> and what does it mean in terms of graph properties ?

shortest paths 
https://github.com/python-graphblas/graphblas-algorithms/blob/main/graphblas_algorithms/algorithms/shortest_paths/generic.py

intro slides
https://apps.dtic.mil/sti/pdfs/AD1083909.pdf


In [ ]:
# reciprocal deg dist based on python-graphblas


hypergraph algorithms for shortest paths, centrality, etc

pdf: centrality_in_social_networks_theor 
